# Baselines + GBM


Baselines: persistence (t‑24) and moving average vs. GBM baseline.

In [ ]:
# Environment setup
import sys, subprocess
from pathlib import Path

print('Python:', sys.executable)
repo_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
subprocess.run(['pip', 'install', '-e', str(repo_root)], check=True)

In [ ]:
import subprocess

subprocess.run([
    'python', '-m', 'gridpulse.data_pipeline.build_features',
    '--in', 'data/raw', '--out', 'data/processed'
], check=True)
subprocess.run([
    'python', '-m', 'gridpulse.data_pipeline.split_time_series',
    '--in', 'data/processed/features.parquet',
    '--out', 'data/processed/splits'
], check=True)


In [ ]:
import subprocess

for target in ['load_mw', 'wind_mw', 'solar_mw']:
    subprocess.run([
        'python', '-m', 'gridpulse.forecasting.train_baseline',
        '--features', 'data/processed/features.parquet',
        '--splits', 'data/processed/splits',
        '--target', target
    ], check=True)


In [ ]:
import json
from pathlib import Path

for p in Path('artifacts/backtests').glob('week1_baseline_*.json'):
    print(p.name)
    print(json.loads(p.read_text()))


## Visual Sanity Checks

These plots provide a quick visual validation of recent behavior.

In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

repo_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
features_path = repo_root / 'data' / 'processed' / 'features.parquet'
if not features_path.exists():
    print('features.parquet not found. Run the feature pipeline first.')
else:
    df_viz = pd.read_parquet(features_path).sort_values('timestamp')
    if {'load_mw','wind_mw','solar_mw'}.issubset(df_viz.columns):
        recent = df_viz.tail(7 * 24)
        fig, ax = plt.subplots(3, 1, figsize=(12, 7), sharex=True)
        recent.plot(x='timestamp', y='load_mw', ax=ax[0], color='#1f77b4', title='Load (last 7 days)')
        recent.plot(x='timestamp', y='wind_mw', ax=ax[1], color='#2ca02c', title='Wind (last 7 days)')
        recent.plot(x='timestamp', y='solar_mw', ax=ax[2], color='#ff7f0e', title='Solar (last 7 days)')
        plt.tight_layout()
    else:
        print('Expected columns not found in features.parquet')
